# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
from datetime import datetime

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weathercsv_df = pd.read_csv('../WeatherPy/output_data/weather_data.csv', index_col=0)

weathercsv_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Turbat,PK,26.0023,63.0440,77.58,16,7,2.84,1613107619
1,Tiksi,RU,71.6872,128.8694,-8.88,89,48,3.98,1613107317
2,Naze,JP,28.3667,129.4833,64.40,82,75,24.16,1613107416
3,Rikitea,PF,-23.1203,-134.9692,79.56,67,98,3.11,1613107240
4,San Francisco,US,37.7749,-122.4194,53.60,87,90,18.41,1613107241
...,...,...,...,...,...,...,...,...,...
545,Luangwa,ZM,-15.6167,30.4167,72.34,91,100,1.90,1613107778
546,Kozhva,RU,65.1073,57.0459,-19.19,87,100,2.66,1613107778
547,Aubenas,FR,44.6201,4.3899,37.00,73,100,2.73,1613107779
548,La Asuncion,VE,11.0333,-63.8628,75.34,85,89,15.26,1613107779


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
max_humid = weathercsv_df['Humidity'].max()

gmaps.configure(api_key=g_key)

locations = weathercsv_df[["Lat", "Lng"]].astype(float)
humidity = weathercsv_df['Humidity']
fig = gmaps.figure(zoom_level=2, center=(15, 0 ), layout={'width': '1100px', 'height': '600px'} )
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_humid,
                                 point_radius=4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='600px', width='1100px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# weathercsv_df.dtypes

hotel_df = weathercsv_df.loc[weathercsv_df['Max Temp'] >= 75]
hotel_df = hotel_df.loc[hotel_df['Max Temp'] <= 100]
hotel_df = hotel_df.loc[hotel_df['Cloudiness'] <= 25]
hotel_df = hotel_df.loc[hotel_df['Wind Speed'] < 10]

hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Turbat,PK,26.0023,63.0440,77.58,16,7,2.84,1613107619
5,Vaini,TO,-21.2000,-175.2000,84.20,84,20,5.75,1613107620
26,Ahwa,IN,20.7500,73.6833,87.06,14,6,2.44,1613107626
35,Richards Bay,ZA,-28.7830,32.0377,75.96,85,8,2.62,1613107369
44,Avarua,CK,-21.2078,-159.7750,84.20,70,24,1.14,1613107264
59,Gigmoto,PH,13.7796,124.3900,83.07,75,17,5.48,1613107637
70,Touros,BR,-5.1989,-35.4608,77.00,88,20,3.44,1613107334
92,Busselton,AU,-33.6500,115.3333,75.24,42,0,3.67,1613106948
103,Tumpat,MY,6.1978,102.1710,87.80,66,20,8.05,1613107649
116,George Town,MY,5.4112,100.3354,91.40,55,20,3.44,1613107264


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df['Hotel Name'] = ''

hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,Turbat,PK,26.0023,63.0440,77.58,16,7,2.84,1613107619,
5,Vaini,TO,-21.2000,-175.2000,84.20,84,20,5.75,1613107620,
26,Ahwa,IN,20.7500,73.6833,87.06,14,6,2.44,1613107626,
35,Richards Bay,ZA,-28.7830,32.0377,75.96,85,8,2.62,1613107369,
44,Avarua,CK,-21.2078,-159.7750,84.20,70,24,1.14,1613107264,
59,Gigmoto,PH,13.7796,124.3900,83.07,75,17,5.48,1613107637,
70,Touros,BR,-5.1989,-35.4608,77.00,88,20,3.44,1613107334,
92,Busselton,AU,-33.6500,115.3333,75.24,42,0,3.67,1613106948,
103,Tumpat,MY,6.1978,102.1710,87.80,66,20,8.05,1613107649,
116,George Town,MY,5.4112,100.3354,91.40,55,20,3.44,1613107264,


In [6]:
search_type = 'hotel'
radius = 5000
params = {
    "radius": radius,
    "types": search_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f'{lat},{lng}'
    
    base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
    
    hotels = requests.get(base_url, params=params)
    
    hotels = hotels.json()
    
    print("Retrieving next hotel")
    
    try:
        print(f'There is a hotel called {hotels["results"][1]["name"]}')
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][1]['name']
        
    except (KeyError, IndexError):
        print('No Hotel Available.')
        hotel_df.loc[index, 'Hotel Name'] = np.nan
        pass
    
    time.sleep(1)

Retrieving next hotel
There is a hotel called Bolan Engineering Workshop
Retrieving next hotel
There is a hotel called Keleti Beach Resort
Retrieving next hotel
There is a hotel called District Panchayat
Retrieving next hotel
There is a hotel called BON Hotel Waterfront Richards Bay
Retrieving next hotel
There is a hotel called Paradise Inn
Retrieving next hotel
There is a hotel called Cj's Travellers Inn
Retrieving next hotel
There is a hotel called INN NEW HORIZON
Retrieving next hotel
There is a hotel called Observatory Guest House
Retrieving next hotel
There is a hotel called Baan Tanyong B&B
Retrieving next hotel
There is a hotel called Cititel Penang
Retrieving next hotel
There is a hotel called HAMDAN PLAZA HOTEL SALALAH
Retrieving next hotel
There is a hotel called Sur Plaza Hotel
Retrieving next hotel
There is a hotel called Waso Secondary School
Retrieving next hotel
There is a hotel called sikndar ali mobile shop
Retrieving next hotel
There is a hotel called OYO 884 Rumoh PM

In [7]:
hotel_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name
0,Turbat,PK,26.0023,63.0440,77.58,16,7,2.84,1613107619,Bolan Engineering Workshop
5,Vaini,TO,-21.2000,-175.2000,84.20,84,20,5.75,1613107620,Keleti Beach Resort
26,Ahwa,IN,20.7500,73.6833,87.06,14,6,2.44,1613107626,District Panchayat
35,Richards Bay,ZA,-28.7830,32.0377,75.96,85,8,2.62,1613107369,BON Hotel Waterfront Richards Bay
44,Avarua,CK,-21.2078,-159.7750,84.20,70,24,1.14,1613107264,Paradise Inn
59,Gigmoto,PH,13.7796,124.3900,83.07,75,17,5.48,1613107637,Cj's Travellers Inn
70,Touros,BR,-5.1989,-35.4608,77.00,88,20,3.44,1613107334,INN NEW HORIZON
92,Busselton,AU,-33.6500,115.3333,75.24,42,0,3.67,1613106948,Observatory Guest House
103,Tumpat,MY,6.1978,102.1710,87.80,66,20,8.05,1613107649,Baan Tanyong B&B
116,George Town,MY,5.4112,100.3354,91.40,55,20,3.44,1613107264,Cititel Penang


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations,
    info_box_content=[f'Hotel Information: {hotel}' for hotel in hotel_info]
)

fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='600px', width='1100px'))